In [2]:
import pandas as pd
import requests

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

In [3]:
gkg = pd.read_csv("data/20240813.gkg.csv", sep="\t")
#gkg.head()

In [4]:
gkg.head()

,DATE,NUMARTS,COUNTS,THEMES,LOCATIONS,PERSONS,ORGANIZATIONS,TONE,CAMEOEVENTIDS,SOURCES,SOURCEURLS
0,20240813,1,NaN,TAX_ETHNICITY;TAX_ETHNICITY_CANADIAN;TAX_FNCAC...,1#Canada#CA#CA#60#-96#CA;1#Israel#IS#IS#31.5#3...,charles krupa;laxman narasimhan;brian niccol;h...,pepsico;starbucks,"2.06185567010309,3.78006872852234,1.7182130584...",1192349779,castanetkamloops.net,https://www.castanetkamloops.net/news/Business...
1,20240813,1,NaN,MANMADE_DISASTER_IMPLIED;,"1#Brazil#BR#BR#-10#-55#BR;4#Sao Paulo, SãPaulo...",thalita valente machado;campos alencar e silva...,NaN,"-1.52542372881356,2.03389830508475,3.559322033...",NaN,bostonglobe.com,https://www.bostonglobe.com/2024/08/12/world/b...
2,20240813,1,NaN,SOC_GENERALCRIME;EPU_CATS_MIGRATION_FEAR_FEAR;...,"2#Indiana, United States#US#USIN#39.8647#-86.2...",shawn scudder;bob courtney,neighborhood watch program;madison police;madi...,"0.199600798403194,1.79640718562874,1.596806387...","1192464356,1192464356,1192501503,1192478508",wdrb.com,https://www.wdrb.com/news/crime-reports/city-o...
3,20240813,1,NaN,TAX_FNCACT;TAX_FNCACT_SPEAKER;UNGP_FORESTS_RIV...,"1#Pakistan#PK#PK#30#70#PK;4#Islamabad, Islamab...",andrei metelitsa,national assembly speaker sardar ayaz sadiq;pa...,"5.64102564102564,5.64102564102564,0,5.64102564...",1192348387,brecorder.com,https://www.brecorder.com/news/40317251/na-spe...
4,20240813,1,NaN,EPU_POLICY;EPU_POLICY_POLITICAL;USPEC_POLITICS...,"3#Washington, Washington, United States#US#USD...",michael l kripchak;michael rulli,penguin club;election commission;johnson leads...,"0.995024875621891,1.82421227197347,0.829187396...",NaN,tribtoday.com,https://www.tribtoday.com/news/local-news/2024...


In [5]:
# create a function to apply to counts to split the string AFFECT#3000000#Americans#1#United States#US#US#39.828175#-98.5795#US; into a list. the separator is ";"

def split_counts(counts, sep=";"):
    if pd.isna(counts):
        return []
    return counts[:-1].split(sep)

# apply the function to the counts column

gkg["counts"] = gkg["COUNTS"].apply(split_counts)
gkg["themes"] = gkg["THEMES"].apply(split_counts)
gkg["locations"] = gkg["LOCATIONS"].apply(split_counts)
gkg["persons"] = gkg["PERSONS"].apply(split_counts)
gkg["organizations"] = gkg["ORGANIZATIONS"].apply(split_counts)
gkg["tone"] = gkg["TONE"].apply(split_counts, sep=",")
gkg["cameoeventids"] = gkg["CAMEOEVENTIDS"].apply(split_counts, sep=",")
gkg["sources"] = gkg["SOURCES"].apply(split_counts)
gkg["sourcesurl"] = gkg["SOURCEURLS"].apply(split_counts, sep='<UDIV>')


In [6]:
def get_dicts_from_lists(counts):
    if len(counts) == 0:
        return []
    return [dict(zip(["COUNTTYPE", "NUMBER", "OBJECTTYPE", "GEO_TYPE", "GEO_FULLNAME", "GEO_COUNTRYCODE", "GEO_ADM1CODE", "GEO_LAT", "GEO_LONG", "GEO_FEATUREID"], i.split("#"))) for i in counts]

gkg["counts"] = gkg["counts"].apply(get_dicts_from_lists)
gkg.loc[:, "first_url"] = gkg["sourcesurl"].apply(lambda x: x[0].strip())

In [7]:
gkg.to_pickle("data/20240813.gkg.pkl")

In [6]:
# expand gkg["sourcesurl"] to create a new row for each value in the list

gkg.explode("sourcesurl")["sourcesurl"].head()

0    https://www.castanetkamloops.net/news/Business...
1    https://www.bostonglobe.com/2024/08/12/world/b...
2    https://www.wdrb.com/news/crime-reports/city-o...
3    https://www.brecorder.com/news/40317251/na-spe...
4    https://www.tribtoday.com/news/local-news/2024...
Name: sourcesurl, dtype: object

In [7]:
url = gkg.explode("sourcesurl")["sourcesurl"].iloc[0]

In [8]:
url = 'https://www.castanetkamloops.net/news/Business/501194/Starbucks-replaces-its-CEO-names-Chipotle-chief-to-head-the-compan'

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ahueteg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from newspaper import Article
from newspaper import Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ahueteg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import time
from contextlib import contextmanager

def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function '{func.__name__}' took {execution_time:.6f} seconds to execute.")
        return result
    return wrapper

@contextmanager
def print_time():
    start_time = time.time()
    yield
    end_time = time.time()
    print(f"Execution time: {end_time - start_time:.6f} seconds")

In [11]:
gkg.loc[:, "first_url"] = gkg["sourcesurl"].apply(lambda x: x[0].strip())

In [7]:
def get_article(url):
    try:
        article = Article(url, config=config)
        article.download()
        article.parse()
        article.nlp()
        data = {
            "title": article.title,
            "text": article.text,
            "summary": article.summary
        }
        return data
    except Exception as e:
        return {
            "title": "ERROR",
            "text": str(e),
            "summary": "ERROR"
        }
# get timing of the function
def get_article_from_list(url_list):
    return get_article(url_list[0])

url = "https://mix967online.iheart.com/content/2024-08-13-katy-perry-orlando-bloom-cross-off-major-bucket-list-item-in-epic-new-clip/"
get_article_from_list([url])

{'title': 'Katy Perry, Orlando Bloom Cross Off Major Bucket-List Item In Epic New Clip',
 'text': 'Katy Perry and Orlando Bloom recently crossed a daring item off their bucket-list and captured detailed footage of the entire adventure.\n\nThe icons enjoyed each other\'s company in Sardinia, Italy over the weekend, and what better way to spend a romantic holiday than by jumping out of a helicopter straight into the ocean? The "Firework" songstress shared video footage to Instagram on Monday (August 12) depicting a successful plunge and memories to last a lifetime.\n\nThe first photo in the epic carousel shows Perry and Bloom mid-jump, bracing for impact of the unsettled water beneath them. The next clip in the series shows what happened before the unforgettable moment they exited the aircraft, both seemingly excited and well-prepared for the jump. After a brief countdown, the pair fearlessly shot through the air and into the ocean, recording their descent as they breached the ocean\'s s

In [25]:
df = gkg.head(1000)

In [31]:
import dask.dataframe as dd
from dask.distributed import Client

ddf = dd.from_pandas(df, npartitions=200)  # Adjust npartitions based on your data size and available resources

# Start a Dask client
client = Client()  # This will use local threads/processes by default

# Apply the function to the 'sourcesurl' column
ddf['articles_data'] = ddf['first_url'].apply(get_article, meta=('articles_data', 'object'))

# Compute the result
result = ddf.compute()

# Close the client
client.close()

/Users/ahueteg/Documents/workspace/ahueteg/hackatons/factored-datathon-2024-ariteam/.venv/lib/python3.12/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [27]:
result.head()

,DATE,NUMARTS,COUNTS,THEMES,LOCATIONS,PERSONS,ORGANIZATIONS,TONE,CAMEOEVENTIDS,SOURCES,SOURCEURLS,counts,themes,locations,persons,organizations,tone,cameoeventids,sources,sourcesurl,first_url,articles_data
0,20240813,1,<NA>,TAX_ETHNICITY;TAX_ETHNICITY_CANADIAN;TAX_FNCAC...,1#Canada#CA#CA#60#-96#CA;1#Israel#IS#IS#31.5#3...,charles krupa;laxman narasimhan;brian niccol;h...,pepsico;starbucks,"2.06185567010309,3.78006872852234,1.7182130584...",1192349779,castanetkamloops.net,https://www.castanetkamloops.net/news/Business...,[],"['TAX_ETHNICITY', 'TAX_ETHNICITY_CANADIAN', 'T...","['1#Canada#CA#CA#60#-96#CA', '1#Israel#IS#IS#3...","['charles krupa', 'laxman narasimhan', 'brian ...","['pepsico', 'starbuck']","['2.06185567010309', '3.78006872852234', '1.71...",['119234977'],['castanetkamloops.ne'],['https://www.castanetkamloops.net/news/Busine...,https://www.castanetkamloops.net/news/Business...,"{'title': 'Starbucks replaces its CEO, names C..."
1,20240813,1,<NA>,MANMADE_DISASTER_IMPLIED;,"1#Brazil#BR#BR#-10#-55#BR;4#Sao Paulo, SãPaulo...",thalita valente machado;campos alencar e silva...,<NA>,"-1.52542372881356,2.03389830508475,3.559322033...",<NA>,bostonglobe.com,https://www.bostonglobe.com/2024/08/12/world/b...,[],['MANMADE_DISASTER_IMPLIED'],"['1#Brazil#BR#BR#-10#-55#BR', '4#Sao Paulo, Sã...","['thalita valente machado', 'campos alencar e ...",[],"['-1.52542372881356', '2.03389830508475', '3.5...",[],['bostonglobe.co'],['https://www.bostonglobe.com/2024/08/12/world...,https://www.bostonglobe.com/2024/08/12/world/b...,{'title': 'Brazil buries pilot who died in pla...
2,20240813,1,<NA>,SOC_GENERALCRIME;EPU_CATS_MIGRATION_FEAR_FEAR;...,"2#Indiana, United States#US#USIN#39.8647#-86.2...",shawn scudder;bob courtney,neighborhood watch program;madison police;madi...,"0.199600798403194,1.79640718562874,1.596806387...","1192464356,1192464356,1192501503,1192478508",wdrb.com,https://www.wdrb.com/news/crime-reports/city-o...,[],"['SOC_GENERALCRIME', 'EPU_CATS_MIGRATION_FEAR_...","['2#Indiana, United States#US#USIN#39.8647#-86...","['shawn scudder', 'bob courtne']","['neighborhood watch program', 'madison police...","['0.199600798403194', '1.79640718562874', '1.5...","['1192464356', '1192464356', '1192501503', '11...",['wdrb.co'],['https://www.wdrb.com/news/crime-reports/city...,https://www.wdrb.com/news/crime-reports/city-o...,{'title': 'City of Madison police launch new n...
3,20240813,1,<NA>,TAX_FNCACT;TAX_FNCACT_SPEAKER;UNGP_FORESTS_RIV...,"1#Pakistan#PK#PK#30#70#PK;4#Islamabad, Islamab...",andrei metelitsa,national assembly speaker sardar ayaz sadiq;pa...,"5.64102564102564,5.64102564102564,0,5.64102564...",1192348387,brecorder.com,https://www.brecorder.com/news/40317251/na-spe...,[],"['TAX_FNCACT', 'TAX_FNCACT_SPEAKER', 'UNGP_FOR...","['1#Pakistan#PK#PK#30#70#PK', '4#Islamabad, Is...",['andrei metelits'],['national assembly speaker sardar ayaz sadiq'...,"['5.64102564102564', '5.64102564102564', '0', ...",['119234838'],['brecorder.co'],['https://www.brecorder.com/news/40317251/na-s...,https://www.brecorder.com/news/40317251/na-spe...,"{'title': 'NA Speaker meets Belarusian envoy',..."
4,20240813,1,<NA>,EPU_POLICY;EPU_POLICY_POLITICAL;USPEC_POLITICS...,"3#Washington, Washington, United States#US#USD...",michael l kripchak;michael rulli,penguin club;election commission;johnson leads...,"0.995024875621891,1.82421227197347,0.829187396...",<NA>,tribtoday.com,https://www.tribtoday.com/news/local-news/2024...,[],"['EPU_POLICY', 'EPU_POLICY_POLITICAL', 'USPEC_...","['3#Washington, Washington, United States#US#U...","['michael l kripchak', 'michael rull']","['penguin club', 'election commission', 'johns...","['0.995024875621891', '1.82421227197347', '0.8...",[],['tribtoday.co'],['https://www.tribtoday.com/news/local-news/20...,https://www.tribtoday.com/news/local-news/2024...,{'title': 'PAC gave more than $7K to YSU boost...


In [16]:
df.loc[:, "article"] = df["sourcesurl"].apply(get_article_from_list)

/var/folders/tf/8v3l8mx557xf5pyvklc18qyr0000gn/T/ipykernel_23941/2647760911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "article"] = df["sourcesurl"].apply(get_article_from_list)


In [28]:
result_df = pd.concat([result.drop('articles_data', axis=1), result['articles_data'].apply(pd.Series)], axis=1)

In [29]:
result_df[["first_url","title", "text", "summary"]].to_pickle("data/20240813.gkg.first-1000-news.pkl")

In [30]:
result_df["title"].value_counts()

title
ERROR                                                                             341
                                                                                    9
International News, Currencies & Economic Indicators                                4
Two men jailed over calls on Facebook to protest outside asylum seekers’ hotel      3
What the papers say – August 13                                                     3
                                                                                 ... 
Mushroom-themed tours emerge as popular summer-vacation option                      1
Nampa Police Post Hilarious Update on Green Power Ranger Arrest                     1
half GGR at Clark casinos up 12pct: Pagcor data                                     1
Vinci Aviation obtains access to a second Learjet 45 - Canadian Manufacturing       1
HGVT Empowers Aspiring Drivers with Comprehensive Lorry Licence Training            1
Name: count, Length: 618, dtype: int64

In [3]:
df = pd.read_pickle("data/20240813.gkg.first-1000-news.pkl")

In [4]:
df.to_csv("data/20240813.gkg.first-1000-news.csv", index=False)